In [1]:
# Import required modules
import numpy as np
import matplotlib.pyplot as plt
from modern_aerospace_propulsion.ffsc_nozzle.cycle import optimize_ffsc_design

print("Modules imported successfully")
print("Note: Requires Cantera, CoolProp, and scipy to be installed")

Modules imported successfully
Note: Requires Cantera, CoolProp, and scipy to be installed


## Example 1: Optimize CH4/O2 Engine for Vacuum Operation

Design a 500 kN thrust methalox engine for vacuum operation (e.g., upper stage).

In [2]:
# Optimize CH4/O2 engine for vacuum
result_vac = optimize_ffsc_design(
    F_vac=500e3,  # 500 kN thrust
    fuel_name="CH4",
    ox_name="O2",
    fuel_coolprop="Methane",
    ox_coolprop="Oxygen",
    T_fuel_tank=110.0,  # K (saturated liquid CH4)
    T_ox_tank=90.0,     # K (saturated liquid O2)
    p_amb=0.0,          # Vacuum
    T_wall_max_limit=2000.0,  # K (copper limit)
    # Bounds for optimization (moderately wide for first attempt)
    p0_bounds=(1e5, 100e6),      # 15-30 MPa chamber pressure
    OF_bounds=(2.0, 4.0),         # O/F ratio range
    r_t_bounds=(0.02, 0.15),      # 80-150 mm throat radius
    eps_bounds=(5.0, 35.0),      # Expansion ratio 15-35
    L_noz_bounds=(0.5, 2.5),      # 1.0-2.5 m nozzle length
    coolant_frac_bounds=(0.1, 0.5),  # 15-45% coolant flow
    # Better initial guess based on typical methalox engines
    initial_guess={
        'p0': 20e6,
        'OF': 3.2,
        'r_t': 0.10,
        'eps': 15.0,
        'L_noz': 1.2,
        'coolant_frac': 0.9,
    },
    # Turbomachinery efficiencies
    eta_pump_fuel=0.70,
    eta_pump_ox=0.70,
    eta_turb_fuel=0.90,
    eta_turb_ox=0.90,
    eta_mech=0.98,
    # Optimizer settings (increased iterations for robustness)
    #optimizer_method="SLSQP",
    optimizer_method="trust-constr",
    max_iterations=10,  # Increased from 100
    verbose=True,
)


Starting FFSC Design Optimization
Target thrust: 500.0 kN
Propellants: CH4/O2
Tank temps: T_fuel=110.0 K, T_ox=90.0 K
Ambient pressure: 0.0 kPa
Wall temp limit: 2000.0 K

Bounds:
  p0:           0.1 - 100.0 MPa
  OF:           2.00 - 4.00
  r_t:          20.0 - 150.0 mm
  eps:          5.0 - 35.0
  L_noz:        0.50 - 2.50 m
  coolant_frac: 10.0% - 50.0%

Initial guess:
  p0=20.0 MPa, OF=3.20, r_t=100.0 mm, eps=15.0, L_noz=1.20 m, coolant_frac=90.0%

--- Testing initial guess ---
  Eval   1: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=    165.25
⚠ WARNING: Initial guess produced invalid result (obj=1.00e+06)
  This may indicate a problem with input parameters or bounds.
  Attempting optimization anyway...


  Eval   2: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=    165.25
  Eval   3: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=    165.25
  Eval   4: ✗ FAILED - ValueError: solver_rho_Tp was unable 

/Users/davidtew/anaconda3/envs/aviation/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py:376: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
/Users/davidtew/anaconda3/envs/aviation/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py:728: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


  Eval  16: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   166.835
  Eval  17: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   166.835
  Eval  18: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   166.835
  Eval  19: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   166.835
  Eval  20: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   166.835
  Eval  21: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   166.835
  Eval  22: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   166.835
  Eval  23: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   167.564
  Eval  24: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   167.564
  Eval  25: ✗ FAILED - ValueError: solver_rho_Tp was unable to find a solution for T=   167.564
  Eval  26: ✗ FAILED - ValueError: solve

In [3]:
# Extract and display optimal results
if result_vac['success']:
    print("\n" + "="*70)
    print("OPTIMAL VACUUM ENGINE DESIGN")
    print("="*70)
    print(f"\nObjective:")
    print(f"  Maximum Isp:       {result_vac['Isp_opt']:.2f} s")
    print(f"\nOptimal Parameters:")
    print(f"  Chamber pressure:  {result_vac['p0_opt']/1e6:.2f} MPa")
    print(f"  O/F ratio:         {result_vac['OF_opt']:.3f}")
    print(f"  Throat radius:     {result_vac['r_t_opt']*1000:.2f} mm")
    print(f"  Expansion ratio:   {result_vac['eps_opt']:.2f}")
    print(f"  Nozzle length:     {result_vac['L_noz_opt']:.3f} m")
    print(f"  Coolant fraction:  {result_vac['coolant_frac_opt']*100:.1f}%")
    print(f"\nPerformance:")
    print(f"  Vacuum thrust:     {result_vac['F_vac']/1e3:.1f} kN")
    print(f"  Total flow:        {result_vac['m_dot_total']:.2f} kg/s")
    print(f"  Fuel flow:         {result_vac['cycle_results']['m_dot_fuel']:.2f} kg/s")
    print(f"  Oxidizer flow:     {result_vac['cycle_results']['m_dot_ox']:.2f} kg/s")
    print(f"\nFeasibility:")
    print(f"  Fuel turbopump:    {'✓ OK' if result_vac['cycle_results']['fuel_side_ok'] else '✗ FAIL'}")
    print(f"  Ox turbopump:      {'✓ OK' if result_vac['cycle_results']['ox_side_ok'] else '✗ FAIL'}")
    print(f"  Max wall temp:     {result_vac['cycle_results']['T_wall_max']:.1f} K")
    print(f"  Coolant used:      {result_vac['cycle_results']['coolant_fraction']*100:.1f}%")
else:
    print("\n" + "="*70)
    print("OPTIMIZATION DID NOT CONVERGE")
    print("="*70)
    print(f"Message: {result_vac['message']}")
    print(f"\nTroubleshooting suggestions:")
    print(f"1. Increase max_iterations to 250-300")
    print(f"2. Try optimizer_method='trust-constr'")
    print(f"3. Relax T_wall_max_limit to 950 K for initial pass")
    print(f"4. Use staged optimization (see Example 5 below)")
    print(f"5. Check if best result during search was close:")
    if result_vac.get('Isp_opt') is not None:
        print(f"   Best Isp found: {result_vac['Isp_opt']:.2f} s")


OPTIMIZATION DID NOT CONVERGE
Message: Constraint violation exceeds 'gtol'

Troubleshooting suggestions:
1. Increase max_iterations to 250-300
2. Try optimizer_method='trust-constr'
3. Relax T_wall_max_limit to 950 K for initial pass
4. Use staged optimization (see Example 5 below)
5. Check if best result during search was close:
   Best Isp found: -inf s


## Example 2: Optimize CH4/O2 Engine for Sea-Level Operation

Design a 1000 kN thrust methalox engine for sea-level operation (e.g., first stage).

In [8]:
# Optimize CH4/O2 engine for sea level
result_sl = optimize_ffsc_design(
    F_vac=1000e3,  # 1000 kN thrust (note: vacuum thrust, sea level will be lower)
    fuel_name="CH4",
    ox_name="O2",
    fuel_coolprop="Methane",
    ox_coolprop="Oxygen",
    T_fuel_tank=110.0,  # K
    T_ox_tank=90.0,     # K
    p_amb=101325.0,     # Sea level (1 atm)
    T_wall_max_limit=900.0,  # K
    # Bounds - for sea level, lower expansion ratio is better
    p0_bounds=(15e6, 35e6),      # Higher pressure helps at sea level
    OF_bounds=(2.5, 4.0),
    r_t_bounds=(0.08, 0.25),
    eps_bounds=(10.0, 25.0),     # Lower expansion for sea level
    L_noz_bounds=(0.5, 2.5),
    # Turbomachinery
    eta_pump_fuel=0.70,
    eta_pump_ox=0.70,
    eta_turb_fuel=0.90,
    eta_turb_ox=0.90,
    eta_mech=0.98,
    # Optimizer
    optimizer_method="SLSQP",
    max_iterations=100,
    verbose=True,
)


Starting FFSC Design Optimization
Target thrust: 1000.0 kN
Propellants: CH4/O2
Tank temps: T_fuel=110.0 K, T_ox=90.0 K
Ambient pressure: 101.3 kPa
Wall temp limit: 900.0 K

Initial guess:
  p0=25.0 MPa, OF=3.25, r_t=165.0 mm, eps=17.5, L_noz=1.50 m, coolant_frac=35.0%


Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 1000000.0
            Iterations: 5
            Function evaluations: 7
            Gradient evaluations: 1

Optimization Complete - FAILED TO CONVERGE
Message: Positive directional derivative for linesearch
Returning best result found during search:



In [ ]:
# Display sea-level optimal results
if result_sl['success']:
    print("\n" + "="*70)
    print("OPTIMAL SEA-LEVEL ENGINE DESIGN")
    print("="*70)
    print(f"\nObjective:")
    print(f"  Maximum Isp (SL): {result_sl['Isp_opt']:.2f} s")
    print(f"\nOptimal Parameters:")
    print(f"  Chamber pressure:  {result_sl['p0_opt']/1e6:.2f} MPa")
    print(f"  O/F ratio:         {result_sl['OF_opt']:.3f}")
    print(f"  Throat radius:     {result_sl['r_t_opt']*1000:.2f} mm")
    print(f"  Expansion ratio:   {result_sl['eps_opt']:.2f}")
    print(f"  Nozzle length:     {result_sl['L_noz_opt']:.3f} m")
    print(f"\nNote: Lower expansion ratio compared to vacuum design")
    print(f"      (avoids over-expansion and flow separation at sea level)")

## Example 3: Compare Vacuum vs. Sea-Level Designs

In [9]:
# Create comparison table
if result_vac['success'] and result_sl['success']:
    import pandas as pd
    
    comparison = pd.DataFrame({
        'Parameter': [
            'Chamber Pressure (MPa)',
            'O/F Ratio',
            'Throat Radius (mm)',
            'Expansion Ratio',
            'Nozzle Length (m)',
            'Isp (s)',
            'Total Flow (kg/s)',
        ],
        'Vacuum Design': [
            f"{result_vac['p0_opt']/1e6:.2f}",
            f"{result_vac['OF_opt']:.3f}",
            f"{result_vac['r_t_opt']*1000:.2f}",
            f"{result_vac['eps_opt']:.2f}",
            f"{result_vac['L_noz_opt']:.3f}",
            f"{result_vac['Isp_opt']:.2f}",
            f"{result_vac['m_dot_total']:.2f}",
        ],
        'Sea-Level Design': [
            f"{result_sl['p0_opt']/1e6:.2f}",
            f"{result_sl['OF_opt']:.3f}",
            f"{result_sl['r_t_opt']*1000:.2f}",
            f"{result_sl['eps_opt']:.2f}",
            f"{result_sl['L_noz_opt']:.3f}",
            f"{result_sl['Isp_opt']:.2f}",
            f"{result_sl['m_dot_total']:.2f}",
        ]
    })
    
    print("\n" + "="*70)
    print("VACUUM vs SEA-LEVEL DESIGN COMPARISON")
    print("="*70)
    print(comparison.to_string(index=False))
    print("\nKey Observations:")
    print("  • Sea-level design has lower expansion ratio (avoids separation)")
    print("  • Sea-level design may use higher chamber pressure")
    print("  • Vacuum design achieves higher Isp due to better expansion")

## Example 4: Sensitivity to Initial Guess

Test if the optimizer is robust to different starting points.

In [10]:
# Try optimization with different initial guesses
initial_guesses = [
    {'p0': 15e6, 'OF': 3.0, 'r_t': 0.10, 'eps': 20.0, 'L_noz': 1.5},
    {'p0': 25e6, 'OF': 3.5, 'r_t': 0.12, 'eps': 25.0, 'L_noz': 2.0},
    {'p0': 20e6, 'OF': 2.8, 'r_t': 0.08, 'eps': 30.0, 'L_noz': 2.5},
]

results_list = []
for i, guess in enumerate(initial_guesses):
    print(f"\n{'='*70}")
    print(f"Trial {i+1}: Initial guess p0={guess['p0']/1e6:.1f} MPa, "
          f"OF={guess['OF']:.2f}, eps={guess['eps']:.1f}")
    print(f"{'='*70}")
    
    result = optimize_ffsc_design(
        F_vac=500e3,
        fuel_name="CH4",
        ox_name="O2",
        fuel_coolprop="Methane",
        ox_coolprop="Oxygen",
        T_fuel_tank=110.0,
        T_ox_tank=90.0,
        p_amb=0.0,
        initial_guess=guess,
        verbose=False,  # Suppress detailed output
    )
    
    results_list.append(result)
    if result['success']:
        print(f"Converged to Isp = {result['Isp_opt']:.2f} s")
    else:
        print(f"Failed to converge: {result['message']}")

# Check if all trials converged to same optimum
if all(r['success'] for r in results_list):
    isp_values = [r['Isp_opt'] for r in results_list]
    print(f"\n{'='*70}")
    print("CONVERGENCE CHECK")
    print(f"{'='*70}")
    print(f"All trials converged successfully")
    print(f"Isp range: {min(isp_values):.2f} - {max(isp_values):.2f} s")
    print(f"Variation: {max(isp_values) - min(isp_values):.3f} s")
    if max(isp_values) - min(isp_values) < 0.5:
        print("✓ Optimizer is robust - all trials found same optimum")
    else:
        print("⚠ Optimizer may have found different local optima")


Trial 1: Initial guess p0=15.0 MPa, OF=3.00, eps=20.0
Failed to converge: Positive directional derivative for linesearch

Trial 2: Initial guess p0=25.0 MPa, OF=3.50, eps=25.0
Failed to converge: Positive directional derivative for linesearch

Trial 3: Initial guess p0=20.0 MPa, OF=2.80, eps=30.0
Failed to converge: Positive directional derivative for linesearch


## Notes on Optimization

### Algorithm Selection
- **SLSQP** (default): Good for smooth problems with both equality and inequality constraints
- **trust-constr**: More robust for difficult constraints, but slower
- **COBYLA**: Derivative-free, good for noisy objectives, but less accurate

### Typical Results for CH4/O2
- **Vacuum Isp**: 330-350 s (depending on constraints)
- **Optimal O/F**: 3.0-3.5 (slightly fuel-rich)
- **Optimal Chamber Pressure**: 20-30 MPa
- **Optimal Expansion Ratio**: 25-40 for vacuum, 12-20 for sea level

### Computational Cost
- Each function evaluation: ~0.5-2 seconds
- Typical convergence: 30-80 evaluations
- Total time: 30-150 seconds per optimization

### Tips for Success
1. Start with reasonable bounds based on propellant properties
2. Use tighter bounds if you know the approximate operating regime
3. For H2/O2, use wider O/F bounds (4-8) as stoichiometric ratio is higher
4. If optimization fails, try relaxing constraints or changing initial guess

## Example 5: Improving Convergence

If optimization doesn't converge, try these strategies:

1. **Increase iterations**: Allow more function evaluations
2. **Better initial guess**: Start closer to the optimum
3. **Relaxed constraints**: Use slightly looser bounds initially
4. **Different optimizer**: Try trust-constr or COBYLA
5. **Staged optimization**: Optimize in steps with tighter bounds each time

In [11]:
# Strategy 1: Increase iterations and use better initial guess
print("Strategy 1: More iterations + better initial guess")
print("="*70)

# Start with a reasonable guess based on typical methalox engines
better_guess = {
    'p0': 22e6,           # ~22 MPa is typical for modern engines
    'OF': 3.3,            # Slightly fuel-rich is common
    'r_t': 0.10,          # Medium throat size
    'eps': 25.0,          # Reasonable vacuum expansion
    'L_noz': 1.8,         # Moderate length
    'coolant_frac': 0.25  # 25% coolant flow typical
}

result_strategy1 = optimize_ffsc_design(
    F_vac=500e3,
    fuel_name="CH4",
    ox_name="O2",
    fuel_coolprop="Methane",
    ox_coolprop="Oxygen",
    T_fuel_tank=110.0,
    T_ox_tank=90.0,
    p_amb=0.0,
    T_wall_max_limit=900.0,
    # More generous bounds
    p0_bounds=(15e6, 30e6),
    OF_bounds=(2.8, 3.8),
    r_t_bounds=(0.08, 0.15),
    eps_bounds=(15.0, 35.0),
    L_noz_bounds=(1.0, 2.5),
    coolant_frac_bounds=(0.15, 0.45),
    # Better initial guess
    initial_guess=better_guess,
    # More iterations
    max_iterations=200,
    optimizer_method="SLSQP",
    verbose=True,
)

if result_strategy1['success']:
    print(f"\n✓ SUCCESS: Isp = {result_strategy1['Isp_opt']:.2f} s")
    print(f"  Coolant fraction: {result_strategy1['coolant_frac_opt']*100:.1f}%")
else:
    print(f"\n✗ Did not converge: {result_strategy1['message']}")

Strategy 1: More iterations + better initial guess

Starting FFSC Design Optimization
Target thrust: 500.0 kN
Propellants: CH4/O2
Tank temps: T_fuel=110.0 K, T_ox=90.0 K
Ambient pressure: 0.0 kPa
Wall temp limit: 900.0 K

Initial guess:
  p0=22.0 MPa, OF=3.30, r_t=100.0 mm, eps=25.0, L_noz=1.80 m, coolant_frac=25.0%


Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 1000000.0
            Iterations: 5
            Function evaluations: 7
            Gradient evaluations: 1

Optimization Complete - FAILED TO CONVERGE
Message: Positive directional derivative for linesearch
Returning best result found during search:


✗ Did not converge: Positive directional derivative for linesearch


In [12]:
# Strategy 2: Use trust-constr optimizer (more robust for constraints)
print("\n\nStrategy 2: trust-constr optimizer")
print("="*70)
print("Note: trust-constr is slower but handles constraints better")

result_strategy2 = optimize_ffsc_design(
    F_vac=500e3,
    fuel_name="CH4",
    ox_name="O2",
    fuel_coolprop="Methane",
    ox_coolprop="Oxygen",
    T_fuel_tank=110.0,
    T_ox_tank=90.0,
    p_amb=0.0,
    T_wall_max_limit=1500.0,
    p0_bounds=(15e6, 30e6),
    OF_bounds=(2.8, 3.8),
    r_t_bounds=(0.08, 0.15),
    eps_bounds=(15.0, 35.0),
    L_noz_bounds=(1.0, 2.5),
    coolant_frac_bounds=(0.15, 0.45),
    initial_guess=better_guess,
    optimizer_method="trust-constr",  # Different optimizer
    max_iterations=150,
    verbose=True,
)

if result_strategy2['success']:
    print(f"\n✓ SUCCESS: Isp = {result_strategy2['Isp_opt']:.2f} s")
    print(f"  Coolant fraction: {result_strategy2['coolant_frac_opt']*100:.1f}%")
else:
    print(f"\n✗ Did not converge: {result_strategy2['message']}")



Strategy 2: trust-constr optimizer
Note: trust-constr is slower but handles constraints better

Starting FFSC Design Optimization
Target thrust: 500.0 kN
Propellants: CH4/O2
Tank temps: T_fuel=110.0 K, T_ox=90.0 K
Ambient pressure: 0.0 kPa
Wall temp limit: 1500.0 K

Initial guess:
  p0=22.0 MPa, OF=3.30, r_t=100.0 mm, eps=25.0, L_noz=1.80 m, coolant_frac=25.0%


  Eval  10: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra


/Users/davidtew/anaconda3/envs/aviation/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py:376: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
/Users/davidtew/anaconda3/envs/aviation/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py:728: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


  Eval  20: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval  30: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval  40: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval  50: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval  60: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval  70: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval  80: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval  90: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval 100: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval 110: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval 120: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval 130: Analysis failed - Inputs in Brent [0.000000,30417.384000] do not bra
  Eval 140: Analysis failed 

In [ ]:
# Strategy 3: Staged optimization (coarse to fine)
print("\n\nStrategy 3: Staged optimization (coarse → fine)")
print("="*70)
print("First pass: Wide bounds, find approximate optimum")

# Stage 1: Wide bounds to find general region
result_stage1 = optimize_ffsc_design(
    F_vac=500e3,
    fuel_name="CH4",
    ox_name="O2",
    fuel_coolprop="Methane",
    ox_coolprop="Oxygen",
    T_fuel_tank=110.0,
    T_ox_tank=90.0,
    p_amb=0.0,
    T_wall_max_limit=950.0,  # Slightly relaxed for first pass
    # Wide bounds
    p0_bounds=(12e6, 32e6),
    OF_bounds=(2.5, 4.0),
    r_t_bounds=(0.07, 0.18),
    eps_bounds=(12.0, 38.0),
    L_noz_bounds=(0.8, 2.8),
    coolant_frac_bounds=(0.12, 0.50),
    optimizer_method="SLSQP",
    max_iterations=100,
    verbose=False,  # Less verbose for first pass
)

if result_stage1['success']:
    print(f"Stage 1 converged: Isp = {result_stage1['Isp_opt']:.2f} s\n")
    
    # Stage 2: Refine around the Stage 1 solution
    print("Second pass: Tighter bounds around Stage 1 optimum")
    
    # Create tighter bounds centered on Stage 1 result
    p0_center = result_stage1['p0_opt']
    OF_center = result_stage1['OF_opt']
    r_t_center = result_stage1['r_t_opt']
    eps_center = result_stage1['eps_opt']
    L_noz_center = result_stage1['L_noz_opt']
    cool_center = result_stage1['coolant_frac_opt']
    
    # ±15% around Stage 1 result
    result_stage2 = optimize_ffsc_design(
        F_vac=500e3,
        fuel_name="CH4",
        ox_name="O2",
        fuel_coolprop="Methane",
        ox_coolprop="Oxygen",
        T_fuel_tank=110.0,
        T_ox_tank=90.0,
        p_amb=0.0,
        T_wall_max_limit=900.0,  # Stricter constraint now
        # Tighter bounds around Stage 1 result
        p0_bounds=(0.85*p0_center, 1.15*p0_center),
        OF_bounds=(0.90*OF_center, 1.10*OF_center),
        r_t_bounds=(0.85*r_t_center, 1.15*r_t_center),
        eps_bounds=(0.85*eps_center, 1.15*eps_center),
        L_noz_bounds=(0.85*L_noz_center, 1.15*L_noz_center),
        coolant_frac_bounds=(0.80*cool_center, min(1.20*cool_center, 0.60)),
        # Use Stage 1 result as initial guess
        initial_guess={
            'p0': p0_center,
            'OF': OF_center,
            'r_t': r_t_center,
            'eps': eps_center,
            'L_noz': L_noz_center,
            'coolant_frac': cool_center,
        },
        optimizer_method="SLSQP",
        max_iterations=100,
        verbose=True,
    )
    
    if result_stage2['success']:
        print(f"\n✓ STAGED OPTIMIZATION SUCCESS")
        print(f"  Final Isp: {result_stage2['Isp_opt']:.2f} s")
        print(f"  Improvement: {result_stage2['Isp_opt'] - result_stage1['Isp_opt']:.2f} s")
        print(f"  Coolant fraction: {result_stage2['coolant_frac_opt']*100:.1f}%")
    else:
        print(f"\n⚠ Stage 2 did not converge, but Stage 1 result available")
        print(f"  Stage 1 Isp: {result_stage1['Isp_opt']:.2f} s")
else:
    print(f"✗ Stage 1 did not converge: {result_stage1['message']}")

## Convergence Troubleshooting Guide

### Common Issues and Solutions:

**1. "Maximum iterations reached"**
- **Solution**: Increase `max_iterations` to 200-300
- **Why**: Complex design space needs more exploration

**2. "Positive directional derivative for linesearch"**
- **Solution**: Try `trust-constr` optimizer instead of SLSQP
- **Why**: SLSQP can struggle with constraint boundaries

**3. Optimizer stops early with poor Isp**
- **Solution**: Provide better `initial_guess` closer to expected result
- **Why**: May be stuck in local minimum

**4. Infeasible constraints**
- **Solution**: Temporarily relax `T_wall_max_limit` (e.g., 950 K) or widen bounds
- **Why**: Design space may be too constrained

**5. Oscillating between designs**
- **Solution**: Use staged optimization (wide → tight bounds)
- **Why**: Fine-tunes solution after finding approximate region

### Recommended Settings by Problem:

**Easy problems** (loose constraints, wide feasible region):
- SLSQP, 100 iterations, default bounds

**Medium problems** (some tight constraints):
- SLSQP, 150-200 iterations, better initial guess

**Difficult problems** (tight constraints, narrow feasible region):
- trust-constr, 200 iterations, staged optimization

**Very difficult problems** (highly constrained):
- Staged optimization with relaxed then tight constraints
- Multiple random initial guesses to find global optimum